In [1]:
import pandas as pd
import numpy as np

# Education Attainment
- <b>Goal</b>: Compile the human capital data from the Wittgenstein Centre. Use "post secondary" education for those 25+


In [17]:
edu_data = pd.read_csv("../../data/global_Education/wicdf (2).csv",skiprows=8)
edu_data

,Area,Year,Age,Education,ISOCode,Distribution
0,Afghanistan,1950,25+,Under 15,4,0.0
1,Afghanistan,1955,25+,Under 15,4,0.0
2,Afghanistan,1960,25+,Under 15,4,0.0
3,Afghanistan,1965,25+,Under 15,4,0.0
4,Afghanistan,1970,25+,Under 15,4,0.0
...,...,...,...,...,...,...
45559,Uruguay,2080,25+,Master and higher,858,1.6
45560,Uruguay,2085,25+,Master and higher,858,1.7
45561,Uruguay,2090,25+,Master and higher,858,1.8
45562,Uruguay,2095,25+,Master and higher,858,1.9


In [11]:
iso_conversion = pd.read_excel("../../data/country_names.xlsx",sheet_name="ISO")
iso_conversion

,Country name,Official state name,Sovereignty,Alpha-2 code,Alpha-3 code,Numeric code
0,Islamic Republic of Afghanistan Afghanistan,The Islamic Republic of Afghanistan,UN member state,AF,AFG,4.0
1,Åland Åland Islands,Åland,Finland,AX,ALA,248.0
2,Albania Albania,The Republic of Albania,UN member state,AL,ALB,8.0
3,Algeria Algeria,The People's Democratic Republic of Algeria,UN member state,DZ,DZA,12.0
4,American Samoa American Samoa,The Territory of American Samoa,United States,AS,ASM,16.0
...,...,...,...,...,...,...
271,Wallis and Futuna Wallis and Futuna,The Territory of the Wallis and Futuna Islands,France,WF,WLF,876.0
272,Western Sahara Western Sahara [aj],The Sahrawi Arab Democratic Republic,Disputed [ak],EH,ESH,732.0
273,Yemen Yemen,The Republic of Yemen,UN member state,YE,YEM,887.0
274,Zambia Zambia,The Republic of Zambia,UN member state,ZM,ZMB,894.0


In [25]:
edu_data_rename = pd.merge(edu_data,iso_conversion[["Alpha-3 code","Numeric code"]],left_on="ISOCode",right_on="Numeric code")
edu_data_select = edu_data_rename.query("Education=='Post Secondary'")
edu_data_select = edu_data_select.drop(columns=["ISOCode","Numeric code","Area","Age","Education"])
edu_data_select = edu_data_select.rename(columns={"Alpha-3 code":"Country Code"})
edu_data_select

,Year,Distribution,Country Code
186,1950,0.0,AFG
187,1955,0.0,AFG
188,1960,0.1,AFG
189,1965,0.2,AFG
190,1970,0.3,AFG
...,...,...,...
45216,2080,26.7,ZWE
45217,2085,28.4,ZWE
45218,2090,30.2,ZWE
45219,2095,32.1,ZWE


In [44]:
ctry = edu_data_select["Country Code"].unique()[0]
for ctry in edu_data_select["Country Code"].unique():
    edu_data_select_ctry = edu_data_select[edu_data_select["Country Code"]==ctry]
    year_range = edu_data_select_ctry["Year"].min()+np.arange(edu_data_select_ctry["Year"].max()-edu_data_select_ctry["Year"].min()+1)
    new_df = pd.DataFrame(data={"year":year_range})
    edu_data_select_ctry = edu_data_select_ctry.set_index("Year")
    new_df = new_df.set_index("year")
    new_df["PostSecondary_Above25"] = edu_data_select_ctry["Distribution"]
    new_df = new_df.interpolate(method='linear', axis=0)
    new_df["Country Code"] = ctry
    if ctry == edu_data_select["Country Code"].unique()[0]:
        full_interpol = new_df
    else:
        full_interpol = pd.concat([full_interpol,new_df])
full_interpol.to_csv("_all_temporal_edu_PostSecondary.csv")
full_interpol

,PostSecondary_Above25,Country Code
year,,
1950,0.00,AFG
1951,0.00,AFG
1952,0.00,AFG
1953,0.00,AFG
1954,0.00,AFG
...,...,...
2096,32.48,ZWE
2097,32.86,ZWE
2098,33.24,ZWE
